<a href="https://colab.research.google.com/github/Bilal-Waleed/PIAIC-Q2-P3/blob/main/PIAIC_Q2_Project_3_LangChain_Function_Tool_Calling_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 3: LangChain Function/Tool Calling Calculator**

In this Project, we will create a simple LangChain Colab Notebook that uses the Google Gemini Flash model to answer user calculation questions. This example below is provided to help you get started assumes you have access to the Gemini API and a basic Python environment. However, you are required to develop and submit your project using Google Colab.

The project Github repo is: https://github.com/panaversity/learn-agentic-ai/tree/main/02_generative_ai_for_beginners/PROJECTS/03_function_calling

In [2]:
!pip install python-dotenv -q
!pip install langchain-community -U -q
!pip install google-generativeai -U -q
!pip install langchain -U -q
!pip install langchain_google_genai -U -q

from google.colab import userdata
import os
import re
import math
from langchain.tools import tool
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [4]:
# Step 1: Retrieve the API key from Colab's user data
api_key = userdata.get('GOOGLE_API_KEY')

# Enhanced Calculator Class
class Calculator:
    def calculate(self, expression: str) -> str:
        # Regex validation for safe mathematical expressions
        if not re.match(r"^[0-9+\-*/()., ]+$", expression):
            return "Error: Invalid characters in expression."

        try:
            # Allow safe functions from math library
            safe_dict = {"__builtins__": None, "math": math}
            result = eval(expression, safe_dict)
            return str(result)
        except ZeroDivisionError:
            return "Error: Division by zero is not allowed."
        except Exception as e:
            return f"Error: {e}"

# Define the calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations, including basic operations and advanced math.
    Input: A mathematical expression as a string (e.g., "2 + 2", "math.sqrt(16)").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

# Initialize the Google Gemini Flash model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=api_key,
    temperature=0.7
)

# Create a list of tools
tools = [calculator]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=gemini_model,
    agent_type="openai-function-calling"
)

# Set up memory for context
memory = ConversationBufferMemory()

# Build the conversational chain
chain = ConversationChain(
    llm=gemini_model,
    memory=memory,
    verbose=True
)

# Example usage
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that.",
    "Calculate the square root of 49.",
    "What is 10 / 0?",  # Test error handling
    "What is log of -1?",  # Invalid input
]

for q in queries:
    print(f"Query: {q}")
    response = chain.invoke(q)
    print(f"Response: {response}")
    print("-" * 40)


Query: What is 25 multiplied by 4?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is 25 multiplied by 4?
AI:

> Finished chain.
Response: {'input': 'What is 25 multiplied by 4?', 'history': '', 'response': "25 multiplied by 4 is 100.  That's the same as adding four 25s together: 25 + 25 + 25 + 25 = 100.  You can also think of it as 25 multiplied by 4, which is 100 because 4 quarters make a dollar, and 25 cents is a quarter, so four quarters make 100 cents, or one dollar."}
----------------------------------------
Query: Now divide the result by 5.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative 